In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
file_path = "/kaggle/input/avazu-ctr-prediction/train.gz"
data_df = pd.read_csv(file_path, header=0, nrows=10000000).sample(frac=1)

In [ ]:
train_count = int(len(data_df) * 0.9)

In [ ]:
X_train = data_df.iloc[:train_count,2:]
y_train = data_df.iloc[:train_count,1]

In [ ]:
X_val = data_df.iloc[train_count:,2:]
y_val = data_df.iloc[train_count:,1]

In [ ]:
X_val.head()

In [ ]:
cat_features = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [ ]:
model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.5,
    task_type='GPU',
    loss_function='Logloss',
#     gpu_ram_part=0.9,
#     boosting_type='Plain',
#     max_ctr_complexity=2,
#     depth=6,
#     gpu_cat_features_storage='CpuPinnedMemory',
)

In [ ]:
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    cat_features=cat_features,
    verbose=10,
)

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
test_file_path = "/kaggle/input/avazu-ctr-prediction/test.gz"
test_df = pd.read_csv(test_file_path, header=0, dtype=str)

In [ ]:
X_test = test_df.iloc[:, 1:]
X_test.head()

In [ ]:
y_test = model.predict(X_test, 
                        prediction_type='Probability', 
                        ntree_start=0, ntree_end=model.get_best_iteration(), 
                        thread_count=-1, verbose=None)

In [ ]:
id_test = test_df.iloc[:, 0:1]
id_test

In [ ]:
id_test.join(pd.DataFrame(y_test)) 

In [ ]:
submission_df = pd.read_csv("/kaggle/input/avazu-ctr-prediction/sampleSubmission.gz")

In [ ]:
submission_df["click"] = y_test[:, 1]
submission_df.to_csv("submission.csv", index=False)
submission_df.head()